# CHUNKING STRATEGY
RULES OF CHUNKING - 
1. Process every document separately.
2. Remove all elements before 'PART I' Business Strategy, 'Table of Contents', and 'Page Numbers'.
3. Chunk all lowest-level headings. Implement a 20% overlap.
4. Merge two chunks if one has less than 200 tokens.
5. Add the full hierarchical path of all headings/subheadings and document metadata to the metadata tag.
(e.g., document_id, fiscal_year, part, section_1, section_2, section_3)
6. Send the chunks along with metadata information to the LLM.
7. Architectural Layer: Implement a Parent-Child Retrieval mechanism where search is done on the fine-grained chunks, but chunks under second-level heading will be combined and sent to LLM.


## 10-K CHUNKING

### Algorithm - 
1. Iterate through every item one by one.
2. Initialize a Dynamic meta data dictionary for every chunk with a 'parent_header' : '{item_no: text}', 'form_type':'10_K', 'Company':{company name}, 'filing_date':'{date}'
3. If any element is a header, mention '--- heading' in front of that and join it back with the parent chunk.
4. Every chunk should contain maximum 250 words.
5. Mention table in the METADATA of table text and keep it a standalone text, no matter what the size is.

### Cleaning Chunks -
1. Remove the footers using this pattern '.* |.* |[0-9]*'
2. Remove the text if they only contain a single number '[0-9]+', it will remove page numbers.
3. Remove any text which contains '[a-z]* inc.' and has less than 3 words, lowercase the text for this.

### Further Refinements (Semantic Chunking) - 
1. For every Heading.
2. Check if the dictionary of headings is empty, if yes append level 1 heading to the dict.
3. Else check for the max heading num, then append it as max + 1.
4. Check if previous chunk was content and has atleast 10 words, then empty the heading dict and append a new heading in level '1' also append the chunk as a new one and empty curr_chunk.
5. While appending content check if the chunk size is 200 words, append if yes. Else continue adding.
6. If consecutively more than 3 headings occur, continue till the next content appears, append all headings as content.

In [1]:
from edgar import set_identity, Filing, find, use_local_storage, set_local_storage_path
import pandas as pd
import re
import os
from dotenv import load_dotenv
from itertools import chain
from copy import deepcopy
import json
import math
from edgar.files.html_documents import TextBlock, TableBlock
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from bs4 import BeautifulSoup
from dateutil.parser import parse

c:\Users\paymo\AppData\Local\pypoetry\Cache\virtualenvs\quantigence-oNV_nXK--py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CHUNK_COUNTER = 1
TENK_TENQ_CHUNKS = []
TENK_CHUNKS = []
TENQ_CHUNKS = []
TOTAL_CHUNKS_TENK = 0
TOTAL_CHUNKS_TENQ = 0
#Reading all the metadata from 10-K
BASE_PATH_TENK = os.getenv("BASE_PATH_TENK")
BASE_PATH_TENQ = os.getenv("BASE_PATH_TENQ")

SAVE_PATH_TENK = os.getenv("OUT_10K")
SAVE_PATH_TENQ = os.getenv("OUT_10Q")
SAVE_PATH_ALL = os.getenv("OUT_ALL")


load_dotenv()
set_identity(os.getenv("SEC_ID"))
use_local_storage(os.getenv("SEC_CACHE"))

In [11]:
OUTPUT_FORMAT = "%Y-%m-%d"

COMPANY_NAMES = [
        "Apple Inc.",
        "MICROSOFT CORP",
        "Alphabet Inc.",
        "AMAZON COM INC.",
        "Meta Platforms, Inc.",
        "NVIDIA CORP",
        "Tesla, Inc.",
        "ORACLE CORP",
        "Salesforce, Inc.",
        "NETFLIX INC",
        "ADOBE INC."
    ]

TICKERS = {
        320193 : "AAPL",
        789019: "MSFT",
        1652044: "GOOGL",
        1018724: "AMZN",
        1326801 : "META",
        1045810: "NVDA",
        1318605 : "TSLA",
        1341439 : "ORCL",
        1108524 : "CRM",
        1065280 : "NFLX",
        796343 : "ADBE"
    }


#Clean Text
def clean_abnormal_text(text):
    # 1. Remove long repetitive sequences (3+ dashes, dots, or symbols)
    # This cleans up the "----" and "......" patterns common in broken tables
    text = re.sub(r'[-._*~=]{3,}', ' ', text)
    
    # 2. Remove "DAU/MAU" noise blocks (appears in facebook)
    # Specifically targets "DAU/MAU:" followed by numbers and percentages
    text = re.sub(r'DAU/MAU:[\d% \-]+', ' ', text)
    
    # 3. Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text) # Normalize all whitespace to single spaces
    
    # 4. Remove isolated clusters of special characters
    # This removes things like "- - - " that linger after other steps
    text = re.sub(r'\s([^\w\s])\s', ' ', text)
    
    return text.strip()



#Post processing the exisiting chunks
def post_processing_chunks(all_chunks):
    global CHUNK_COUNTER
    all_chunks_processed = all_chunks.copy()
    splitter = SentenceSplitter(
                chunk_size=300,
                chunk_overlap=60
            )
    for chunk in all_chunks_processed:
        chunk['ID'] = CHUNK_COUNTER
        CHUNK_COUNTER+=1
        if chunk["Metadata"]:
            text = chunk["Chunks"]["parent_chunk"]
            chunk["Chunks"]['child_chunks'] = {}

            if chunk["Metadata"]['is_table']:
                chunk["Chunks"]['child_chunks'][f"{chunk['ID']}_{0}"] = text
                continue
            
            text  = clean_abnormal_text(text)
            documents = [Document(text = text)]
            nodes = splitter.get_nodes_from_documents(documents)

            for i,node in enumerate(nodes):
                child_chunk_dict = chunk["Chunks"]['child_chunks']
                if i==0:
                    child_chunk_dict[f"{chunk['ID']}_{i}"]=node.text
                elif i>0:
                    if len(node.text.split()) > 50:
                        child_chunk_dict[f"{chunk['ID']}_{i}"]=node.text
                    elif node.text not in child_chunk_dict[list(child_chunk_dict)[-1]]:
                        child_chunk_dict[list(child_chunk_dict)[-1]]+=node.text
                        
    return all_chunks_processed


#We will is_table as true if 30% of all_items matches one of the tables
def get_table_of_contents(chunk_obj):
    def is_table(table,all_items):
        q_items = all_items[:math.ceil(len(all_items)*0.3)]
        table = table.get_text().lower()
        for item in q_items:
            if item.lower() not in table:
                return False
        return True
    
    all_items = chunk_obj.list_items()
    table_of_contents = pd.DataFrame()
    MAX_ITEMS = -1
    ITEM_COLUMN_INDEX = -1
    for table in chunk_obj.tables():
        if is_table(table, all_items):
            table_of_contents = table.to_dataframe()
            #Naming the 'Items' columns and 'Headings' columns
            for i in range(table_of_contents.shape[1]):
                NUM_ITEMS = sum('item' in item.lower() for item in table_of_contents.iloc[:,i])
                if NUM_ITEMS > MAX_ITEMS:
                    ITEM_COLUMN_INDEX = i
                    MAX_ITEMS = NUM_ITEMS
            
            #naming the columns
            columns = [''] * table_of_contents.shape[1]
            columns[ITEM_COLUMN_INDEX] = 'Index'
            columns[ITEM_COLUMN_INDEX + 1] = 'Headings'

            #setting the columns to datarame's column att
            table_of_contents.columns = columns
            table_of_contents = table_of_contents.set_index('Index')

            return table_of_contents
        
def get_heading_dict(all_items, table_of_contents):
    #Dictionary of items
    dict = {}

    #if table of contents is none, then we will return items themselves in keys and value both
    if table_of_contents.empty :
        for item in all_items:
            dict[item] = item
        return dict

    for item in all_items:
        for t_item in table_of_contents.index:
            if bool(re.fullmatch(item.lower() + '\W*',t_item.lower())):
                dict[item] = item + ': ' + table_of_contents.loc[t_item,'Headings']
                break
    return dict

#Create chunk function
def emit_chunk(all_chunks, metadata, text):
    if len(text.split()) > 20:
        all_chunks.append({
            "Metadata": deepcopy(metadata),
            "Chunks": {"parent_chunk": text}
        })
    metadata.pop("sub_headings", None)
    metadata["is_table"] = False
    return all_chunks, metadata

def chunk_document(chunk_obj, filing_date, company, item_heading_dictionary,accession_no, form_type = '10-K'):
    regex1 = re.compile(
                    r"""
                        \n*                     # optional newlines at start
                        .*?                     # company name
                        \s\|\s                  # ' | '
                        .*?                     # Any text like 'Q2'
                        \d{4}                   # 4-digit year
                        \sForm\s10-K            # ' Form 10-K'
                        \s\|\s                  # ' | '
                        \d+                     # page number
                        \n*                     # optional newlines at end
                        $""",
                    re.VERBOSE,
                    )
    # Regex 2: Matches only digits
    # ^\d+$
    regex2 = re.compile(r"\n*\d+\n*$")

    metadata = {'item_heading':None, 'filing_date':filing_date, 'company':company, 'form':form_type, 'is_table':False,"accession_no":accession_no}
    #Get all the items present in the table
    items = chunk_obj.list_items()

    all_chunks = []
    processed_chunk = []
    count_of_words = 0
    for item in items:
        metadata['item_heading'] = item_heading_dictionary.get(item, item)
        chunks_for_item = chunk_obj.chunks_for_item(item)
        Headings = {}
        prev_content = False
        prev_heading = False
        metadata = metadata.copy()
        count_of_words = 0
        for chunk in chunks_for_item:
            for element in chunk:
                #We want only text or tables
                if isinstance(element, (TextBlock)):
                    if element.is_header:
                        if regex1.fullmatch(element.get_text()) or regex2.fullmatch(element.get_text()):
                            Headings = {}
                            continue

                        text = element.get_text().strip('\n').strip(' ')
                        if item.lower() in text.lower() or 'part' in text.lower():
                            continue
                        if prev_content and processed_chunk:
                            chunk_text = ' '.join(processed_chunk)
                            count_of_words = 0
                            if len(chunk_text.split()) > 20:
                                all_chunks,metadata = emit_chunk(all_chunks, metadata, chunk_text)
                            processed_chunk = []
                            Headings = {}
                        
                        if Headings:
                            max_level = max(list(Headings.keys()))
                            Headings[max_level+1] = text
                        else:
                            Headings[1] = text
                        
                        prev_content = False
                        prev_heading = True
                    
                    else:
                        if prev_heading:
                            if len(Headings) > 3:
                                chunk_text = ''
                                for key, value in Headings.items():
                                    chunk_text+=value + '\n'
                                all_chunks,metadata = emit_chunk(all_chunks, metadata, chunk_text)
                            else:
                                metadata['sub_headings'] = {}
                                for key, value in Headings.items():
                                    metadata['sub_headings']['level_'+str(key)+'_heading'] = value

                        text = element.get_text().strip('\n').strip(' ')
                        processed_chunk.append(text)
                        count_of_words += len(text.split())
                        prev_content = True
                        prev_heading = False

                elif isinstance(element, TableBlock):
                    if prev_heading:
                        if len(Headings) > 3:
                            chunk_text = ''
                            for key, value in Headings.items():
                                chunk_text+=value + '\n'
                            all_chunks,metadata = emit_chunk(all_chunks, metadata, chunk_text)
                        else:
                            metadata['sub_headings'] = {}
                            for key, value in Headings.items():
                                metadata['sub_headings']['level_'+str(key)+'_heading'] = value
                    
                    chunk_text = ''
                    if prev_content:
                        text = '\n'.join(processed_chunk)
                        if len(text.split()) < 20 and len(text.split()) > 5:
                            chunk_text = text
                            processed_chunk = []

                    chunk_text += element.get_text()
                    metadata['is_table'] = True
                    all_chunks,metadata = emit_chunk(all_chunks, metadata, chunk_text)
                    prev_content = True
                    prev_heading = False

        if processed_chunk:
            count_of_words = 0
            chunk_text = '\n'.join(processed_chunk)
            if len(chunk_text.split()) > 20:
                all_chunks,metadata = emit_chunk(all_chunks, metadata, chunk_text)
            processed_chunk = []

    return all_chunks


def get_chunks(filing_date: str,ticker:str, company: str, cik: int, accession_no:str) -> list:
    filing = Filing(
            form='10-K',
            filing_date= filing_date,
            company= ticker,
            cik=cik,
            accession_no= accession_no
        )

    obj = filing.obj()
    chunk_obj = obj.chunked_document

    table_of_contents = get_table_of_contents(chunk_obj)
    item_heading_dictionary = get_heading_dict(sorted(chunk_obj.list_items()), table_of_contents)
    all_chunks = chunk_document(chunk_obj, filing_date, company, item_heading_dictionary,accession_no, form_type = '10-K')
    
    print(f"Processed {ticker}, {filing_date}")
    return all_chunks

def main():
    global TOTAL_CHUNKS_TENK
    for (cik, ticker),company in zip(TICKERS.items(),COMPANY_NAMES):
        meta_df_path = os.path.join(BASE_PATH_TENK, ticker, '10-K.csv')
        meta_df = pd.read_csv(meta_df_path)
        meta_df = meta_df[pd.to_datetime(meta_df['filing_date']).dt.year > 2023]

        for _, row in meta_df.iterrows():
            filing_date = row['filing_date']
            accession_no = row['accession_number']
            # Convert date to standard format
            datetime_obj  = parse(filing_date)
            filing_date = datetime_obj.strftime(OUTPUT_FORMAT)
            
            all_chunks = get_chunks(filing_date, ticker, company, cik, accession_no)
            TOTAL_CHUNKS_TENK+=len(all_chunks)
            all_chunks_processed = post_processing_chunks(all_chunks)
            
            chunk_det = {
                'filing_date' : filing_date,
                'ticker' : ticker,
                'file_chunks' : all_chunks_processed
            }
            
            TENK_CHUNKS.append(chunk_det)
if __name__ == '__main__':
    main()

Processed AAPL, 2025-10-31
Processed AAPL, 2024-11-01
Processed MSFT, 2025-07-30
Processed MSFT, 2024-07-30
Processed GOOGL, 2025-02-05
Processed GOOGL, 2024-01-31
Processed AMZN, 2025-02-07
Processed AMZN, 2024-02-02
Processed META, 2025-01-30
Processed META, 2024-02-02
Processed NVDA, 2025-02-26
Processed NVDA, 2024-02-21
Processed TSLA, 2025-04-30
Processed TSLA, 2025-01-30
Processed TSLA, 2024-01-29
Processed ORCL, 2025-06-18
Processed ORCL, 2024-06-20
Processed CRM, 2025-03-05
Processed CRM, 2024-03-06
Processed NFLX, 2025-01-27
Processed NFLX, 2024-01-26
Processed ADBE, 2025-01-13
Processed ADBE, 2024-01-17


### SAVE 10K Chunks

In [12]:
import json

# Use 'w' mode for writing text
with open(SAVE_PATH_TENK, "w", encoding="utf-8") as f:
    json.dump(TENK_CHUNKS, f, indent=4)

## 10-Q CHUNKING

### Algorithm - 
1. Iterate over Parts one by one.
2. Check in headings and tables for 'Item' headings and match with the Parts' items.
3. Initialize a Dynamic meta data dictionary for every chunk with a 'part_header' : '{part_no, headings_text} and 'parent_header' : '{item_no, heading_text}', 'form_type':'10_K', 'Company':{company name}, 'filing_date':'{date}'
3. If any element is a header, mention '--- heading' in front of that and join it back with the parent chunk.
4. Every chunk should contain maximum 250 words.
5. Every table should be a separate chunk.
6. Mention table in the METADATA of table text and keep it a standalone text, no matter what the size is.

### Cleaning Chunks.
1. Remove the footers using this pattern '.* |.* |[0-9]*'
2. Remove the text if they only contain a single number '[0-9]+', it will remove page numbers.
3. Remove any text which contains '[a-z]* inc.' and has less than 3 words, lowercase the text for this.

#### NOTE: NETFLIX 10-Q files does not contain "PART I", that is not handled in current code. It has to be handled in next version

In [13]:
metadata_csv = pd.DataFrame()

OUTPUT_FORMAT = "%Y-%m-%d"

COMPANY_NAMES = [
        "Apple Inc.",
        "MICROSOFT CORP",
        "Alphabet Inc.",
        "AMAZON COM INC.",
        "Meta Platforms, Inc.",
        "NVIDIA CORP",
        "Tesla, Inc.",
        "ORACLE CORP",
        "Salesforce, Inc.",
        "NETFLIX INC",
        "ADOBE INC."
    ]

TICKERS = {
        320193 : "AAPL",
        789019: "MSFT",
        1652044: "GOOGL",
        1018724: "AMZN",
        1326801 : "META",
        1045810: "NVDA",
        1318605 : "TSLA",
        1341439 : "ORCL",
        1108524 : "CRM",
        1065280 : "NFLX",
        796343 : "ADBE"
    }

#Clean Text
def clean_abnormal_text(text):
    # 1. Remove long repetitive sequences (3+ dashes, dots, or symbols)
    # This cleans up the "----" and "......" patterns common in broken tables
    text = re.sub(r'[-._*~=]{3,}', ' ', text)
    
    # 2. Remove "DAU/MAU" noise blocks
    # Specifically targets "DAU/MAU:" followed by numbers and percentages
    text = re.sub(r'DAU/MAU:[\d% \-]+', ' ', text)
    
    # 3. Remove excessive whitespace
    # Keep standard spaces but remove non-standard ones
    text = re.sub(r'\s+', ' ', text) # Normalize all whitespace to single spaces
    
    # 4. Remove isolated clusters of special characters
    # This removes things like "- - - " that linger after other steps
    text = re.sub(r'\s([^\w\s])\s', ' ', text)
    
    return text.strip()

def check_and_restore(gen):
    try:
        first_item = next(gen)
        # Re-construct the generator by prepending the item we just took
        return False, chain([first_item], gen)
    except StopIteration:
        return True, gen


#Post processing the exisiting chunks
def post_processing_chunks(all_chunks):
    global CHUNK_COUNTER
    all_chunks_processed = all_chunks.copy()
    splitter = SentenceSplitter(
                chunk_size=300,
                chunk_overlap=60
            )
    for chunk in all_chunks_processed:
        chunk['ID'] = CHUNK_COUNTER
        CHUNK_COUNTER+=1
        if chunk["Metadata"]:
            text = chunk["Chunks"]["parent_chunk"]
            chunk["Chunks"]['child_chunks'] = {}

            if chunk["Metadata"]['is_table']:
                chunk["Chunks"]['child_chunks'][f"{chunk['ID']}_{0}"] = text
                continue
            
            text  = clean_abnormal_text(text)
            documents = [Document(text = text)]
            nodes = splitter.get_nodes_from_documents(documents)

            for i,node in enumerate(nodes):
                child_chunk_dict = chunk["Chunks"]['child_chunks']
                if i==0:
                    child_chunk_dict[f"{chunk['ID']}_{i}"]=node.text
                elif i>0:
                    if len(node.text.split()) > 50:
                        child_chunk_dict[f"{chunk['ID']}_{i}"]=node.text
                    elif node.text not in child_chunk_dict[list(child_chunk_dict)[-1]]:
                        child_chunk_dict[list(child_chunk_dict)[-1]]+=node.text
                        
    return all_chunks_processed

#We will is_table as true if 30% of all_items matches one of the tables
def get_table_of_contents(chunk_obj):
    def is_table(table,all_items):
        q_items = all_items[:math.ceil(len(all_items)*0.3)]
        table = table.get_text().lower()
        for item in q_items:
            if item.lower() not in table:
                return False
        return True
    
    all_items = chunk_obj.list_items()
    table_of_contents = pd.DataFrame()
    MAX_ITEMS = -1
    ITEM_COLUMN_INDEX = -1
    for table in chunk_obj.tables():
        if is_table(table, all_items):
            table_of_contents = table.to_dataframe()
            
            #Naming the 'Items' columns and 'Headings' columns
            for i in range(table_of_contents.shape[1]):
                NUM_ITEMS = sum('item' in item.lower() for item in table_of_contents.iloc[:,i])
                if NUM_ITEMS > MAX_ITEMS:
                    ITEM_COLUMN_INDEX = i
                    MAX_ITEMS = NUM_ITEMS
            
            #Naming the columns
            columns = [''] * table_of_contents.shape[1]
            columns[ITEM_COLUMN_INDEX] = 'Index'
            columns[ITEM_COLUMN_INDEX + 1] = 'Headings'

            #Setting the columns to datarame's column attribute
            table_of_contents.columns = columns
            
            return table_of_contents

def get_correct_item(item, all_items):
    for actual_item in all_items:
        if re.fullmatch(item+'[^a-zA-Z0-9]*',actual_item):
            return actual_item
    
    item = item.strip('. ').split()
    item = item[0].capitalize() + ' ' + item[1]
    return item



def get_heading_dict(all_items, table_of_contents):
    #Dictionary of items
    headings = {}
    part_1 = False
    part_2 = False

    #if table of contents is none, then we will return items themselves in keys and value both
    if table_of_contents.empty :
        headings['Part I'] = {'Heading':'Financial Information','Items':[]}
        headings['Part II'] = {'Heading':'Other Information','Items':[]}
        return headings

    for _, row in table_of_contents.iterrows() :
        if re.fullmatch(r'part i[^a-zA-Z0-9]*', row['Index'].lower()) or 'financial information' in row['Index'].lower() and not part_1:
            headings['Part I'] = {'Heading':'Part I: Financial Information','Items':{}}
            part_1, part_2 = True, False
            
        elif re.fullmatch(r'part i[^a-zA-Z0-9]*', row['Headings'].lower()) or 'financial information' in row['Headings'].lower() and not part_1:
            headings['Part I'] = {'Heading':'Part I: Financial Information','Items':{}}
            part_1, part_2 = True, False

        elif re.fullmatch(r'part ii[^a-zA-Z0-9]*', row['Index'].lower()) or 'other information' in row['Index'].lower() and not part_2:
            headings['Part II'] = {'Heading':'Part II: Other Information','Items':{}}
            part_1, part_2 = False, True

        elif re.fullmatch(r'part ii[^a-zA-Z0-9]*', row['Headings'].lower()) or 'other information' in row['Headings'].lower() and not part_2:
            headings['Part II'] = {'Heading':'Part II: Other Information','Items':{}}
            part_1, part_2 = False, True

        if part_1:
            item = row['Index'].strip('.')
            heading = row['Headings']
            if 'item' in item.lower():
                actual_item = get_correct_item(item,all_items)
                headings['Part I']['Items'][actual_item] = actual_item + ': '+ heading.strip(':. \n')

        elif part_2:
            item = row['Index'].strip('.')
            heading = row['Headings']
            if 'item' in item.lower():
                actual_item = get_correct_item(item,all_items)
                headings['Part II']['Items'][actual_item] = actual_item + ': '+ heading.strip(':. \n')

    return headings
    
#Create chunk function
def emit_chunk(all_chunks, metadata, text):
    if len(text.split()) > 20:
        all_chunks.append({
            "Metadata": deepcopy(metadata),
            "Chunks": {"parent_chunk": text}
        })
    metadata.pop("sub_headings", None)
    metadata["is_table"] = False
    return all_chunks, metadata

def chunk_document(chunk_obj, filing_date, company, heading_dictionary,accession_no, form_type = '10-Q'):

    regex1 = re.compile(
                        r"""
                            \n*                     # optional newlines at start
                            .*?                     # company name
                            \s\|\s                  # ' | '
                            .*?                     # Any text like 'Q2'
                            \d{4}                   # 4-digit year
                            \sForm\s10-Q            # ' Form 10-K'
                            \s\|\s                  # ' | '
                            \d+                     # page number
                            \n*                     # optional newlines at end
                            $""",
                        re.VERBOSE,
                        )

    # Regex 2: Matches only digits
    # ^\d+$
    regex2 = re.compile(r"^\n*\d+\n*$")

    metadata = {'part_heading':None,'item_heading':None, 'filing_date':filing_date, 'company':company, 'form':form_type, 'is_table':False,"accession_no":accession_no}
    #Get all the items present in the table

    all_chunks = []

    processed_chunk = []
    prev_heading = {'T/F':False,'Heading': None}
    count_of_words = 0
    
    for part in ['Part I','Part II']:
        metadata['part_heading'] = heading_dictionary[part]['Heading']
        part_items = list(heading_dictionary[part]['Items'].keys())
       
        chunk_for_part = chunk_obj.chunks_for_part(part)
        
        Headings = {}
        prev_content = False
        prev_heading = False
        count_of_words = 0
        
        for chunk in chunk_for_part:
            for element in chunk:
                #We want only text or tables
                if isinstance(element, (TextBlock)):
                    if element.is_header:
                        #If header contains any of the following should be discarded
                        if regex1.fullmatch(element.get_text()) or regex2.fullmatch(element.get_text()) or 'table of content' in element.get_text().lower():
                            continue

                        #Extracting Items                        
                        for item in part_items:
                            item_heading = re.sub("item\s*\d*\w*:\s*","",heading_dictionary[part]['Items'][item].lower())
                            if item_heading in element.get_text().lower():
                                #Clearing of any existing chunk
                                if processed_chunk:
                                    count_of_words = 0
                                    chunk_text = '\n'.join(processed_chunk)
                                    if len(chunk_text.split()) > 20:
                                        all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
                                    processed_chunk = []
                                #Updating the new item heading
                                metadata['item_heading'] = heading_dictionary[part]['Items'][item]
                        
                        #Now retrieving text for processing
                        text = element.get_text().strip('\n').strip(' ')

                        #If text contains 'item' or 'part' it should be discarded.
                        #As it has been already taken care of
                        if 'item' in text.lower() or 'part' in text.lower():
                            continue
                        
                        #If previous element was content then processed chunk has to be added or flushed
                        if prev_content and processed_chunk:
                            chunk_text = ' '.join(processed_chunk)
                            count_of_words = 0
                            if len(chunk_text.split()) > 20:
                                all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
                            processed_chunk = []
                            Headings = {}
                        
                        #Appending the next heading in existing dict
                        if Headings:
                            max_level = max(list(Headings.keys()))
                            Headings[max_level+1] = text
                        #Creating a new dictionary
                        else:
                            Headings[1] = text
                        
                        #Setting previous content to False and preious heading to True
                        prev_content = False
                        prev_heading = True
                    
                    #If the element is not header
                    else:
                        if prev_heading:
                            #If length of heading is 
                            if len(Headings) > 3:
                                chunk_text = ''
                                for key, value in Headings.items():
                                    chunk_text+=value + '\n'
                                all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)

                            #If length is not more than 3 create sub headings dictionary
                            else:
                                metadata['sub_headings'] = {}
                                for key, value in Headings.items():
                                    metadata['sub_headings']['level_'+str(key)+'_heading'] = value

                        text = element.get_text().strip('\n').strip(' ')
                        processed_chunk.append(text)
                        count_of_words += len(text.split())
                        prev_content = True
                        prev_heading = False

                elif isinstance(element, TableBlock):
                    #Update Metadata parent_heading key
                    for item in part_items:
                        item_heading = re.sub("item\s*\d*\w*:\s*","",heading_dictionary[part]['Items'][item].lower())
                        if item_heading in element.get_text().lower():
                            if processed_chunk:
                                chunk_text = '\n'.join(processed_chunk)
                                if len(chunk_text.split()) > 20:
                                    all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
                                processed_chunk = []
                                count_of_words = 0
                            metadata['item_heading'] = heading_dictionary[part]['Items'][item]

                    #Eliminating the table of contents by checking for number of 'item' in text
                    if element.get_text().lower().count('item') < 6:
                        if prev_heading or prev_content:
                            if len(Headings) > 3:
                                chunk_text = ''
                                for key, value in Headings.items():
                                    chunk_text+=value + '\n'
                                all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
                            else:
                                metadata['sub_headings'] = {}
                                for key, value in Headings.items():
                                    metadata['sub_headings']['level_'+str(key)+'_heading'] = value

                        chunk_text = ''
                        if prev_content:
                            text = '\n'.join(processed_chunk)
                            if len(text.split()) < 20 and len(text.split()) > 5:
                                chunk_text = text
                                processed_chunk = []

                        chunk_text += element.get_text()
                        
                        metadata['is_table'] = True
                        all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
                        prev_content = True
                        prev_heading = False

        if processed_chunk:
            count_of_words = 0
            chunk_text = '\n'.join(processed_chunk)
            if len(chunk_text.split()) > 20:
                all_chunks, metadata = emit_chunk(all_chunks, metadata, chunk_text)
            processed_chunk = []
    
    return all_chunks


def get_chunks(filing_date: str,ticker:str, company: str, cik: int, accession_no:str) -> list:
    # print(filing_date, ticker, cik, accession_no)
    filing = Filing(
            form='10-Q',
            filing_date= filing_date,
            company= ticker,
            cik=cik,
            accession_no= accession_no
        )

    obj = filing.obj()
    chunk_obj = obj.chunked_document

    table_of_contents = get_table_of_contents(chunk_obj)
    heading_dictionary = get_heading_dict(sorted(chunk_obj.list_items()),table_of_contents)
    all_chunks = chunk_document(chunk_obj, filing_date, company, heading_dictionary, accession_no, form_type = '10-Q')
    
    print(f"Processed {ticker}, {filing_date}")
    return all_chunks

def main():
    global TOTAL_CHUNKS_TENQ
    for (cik, ticker),company in zip(TICKERS.items(),COMPANY_NAMES):
        meta_df_path = os.path.join(BASE_PATH_TENQ, ticker, '10-Q.csv')
        meta_df = pd.read_csv(meta_df_path)
        meta_df = meta_df[pd.to_datetime(meta_df['filing_date']).dt.year > 2023]
        for _, row in meta_df.iterrows():
            filing_date = row['filing_date']
            accession_no = row['accession_number']

            # Convert date to standard format
            datetime_obj  = parse(filing_date)
            filing_date = datetime_obj.strftime(OUTPUT_FORMAT)
            
            all_chunks = get_chunks(filing_date, ticker, company, cik, accession_no)
            TOTAL_CHUNKS_TENQ+=len(all_chunks)
            all_chunks_processed = post_processing_chunks(all_chunks)
            
            chunk_det = {
                'filing_date' : filing_date,
                'ticker' : ticker,
                'file_chunks' : all_chunks_processed
            }
            TENQ_CHUNKS.append(chunk_det)

if __name__ == '__main__':
    main()

Processed AAPL, 2025-08-01
Processed AAPL, 2025-05-02
Processed AAPL, 2025-01-31
Processed AAPL, 2024-08-02
Processed AAPL, 2024-05-03
Processed AAPL, 2024-02-02
Processed MSFT, 2025-10-29
Processed MSFT, 2025-04-30
Processed MSFT, 2025-01-29
Processed MSFT, 2024-10-30
Processed MSFT, 2024-04-25
Processed MSFT, 2024-01-30
Processed GOOGL, 2025-10-30
Processed GOOGL, 2025-07-24
Processed GOOGL, 2025-04-25
Processed GOOGL, 2024-10-30
Processed GOOGL, 2024-07-24
Processed GOOGL, 2024-04-26
Processed AMZN, 2025-10-31
Processed AMZN, 2025-08-01
Processed AMZN, 2025-05-02
Processed AMZN, 2024-11-01
Processed AMZN, 2024-08-02
Processed AMZN, 2024-05-01
Processed META, 2025-10-30
Processed META, 2025-07-31
Processed META, 2025-05-01
Processed META, 2024-10-31
Processed META, 2024-08-01
Processed META, 2024-04-25
Processed NVDA, 2025-11-19
Processed NVDA, 2025-08-27
Processed NVDA, 2025-05-28
Processed NVDA, 2024-11-20
Processed NVDA, 2024-08-28
Processed NVDA, 2024-05-29
Processed TSLA, 2025-1

### SAVE 10-Q CHUNKS

In [14]:
import json

# Use 'w' mode for writing text
with open(SAVE_PATH_TENQ, "w", encoding="utf-8") as f:
    json.dump(TENQ_CHUNKS, f, indent=4)

## Save all Chunks

In [15]:
tenk_tenq_chunks = TENK_CHUNKS + TENQ_CHUNKS

with open(SAVE_PATH_ALL,'w', encoding="utf-8") as f:
    json.dump(tenk_tenq_chunks, f, indent = 4)

## Saving all transcripts into one

In [3]:
import json
from dotenv import load_dotenv
import os

load_dotenv()
CHUNK_ID = os.getenv("LATEST_CHUNK_ID")
CHUNK_ID = int(CHUNK_ID)+1

In [5]:

TICKERS = {
    "AAPL": "Apple Inc.",
    "MSFT": "MICROSOFT CORP",
    "GOOGL": "Alphabet Inc.",
    "AMZN": "AMAZON COM INC.",
    "META": "Meta Platforms, Inc.",
    "NVDA": "NVIDIA CORP",
    "TSLA": "Tesla, Inc.",
    "ORCL": "ORACLE CORP",
    "CRM": "Salesforce, Inc.",
    "NFLX": "NETFLIX INC",
    "ADBE": "ADOBE INC."
}
years = ["2024","2025"]
quarters = ['Q1','Q2','Q3']

base_path = "C:/Users/paymo/Downloads/QuantiGence/data/metadata/10-Q/{ticker}/earnings_transcript_{year}{quarter}.json"
save_path = "C:/Users/paymo/Downloads/QuantiGence/Chunks/all_transcripts.json"
all_transcripts = []

#Post processing the exisiting chunks
def post_processing_chunks(all_chunks,company,year,quarter):
    global CHUNK_ID
    all_chunks_copy = all_chunks.copy()
    splitter = SentenceSplitter(
                chunk_size=300,
                chunk_overlap=60
            )
    all_chunks_processed = []
    for chunk in all_chunks_copy["transcript"]:
        metadata = {}
        metadata["speaker"] = chunk["speaker"]
        metadata["title"] = chunk["title"]
        metadata["company"] = company
        metadata["year"] = year
        metadata["quarter"] = quarter

        parent_chunk = {}
        parent_chunk["ID"] = CHUNK_ID
        parent_chunk["chunk"] = chunk["content"]

        child_chunks = {}
        if metadata:
            text = parent_chunk["chunk"]

            documents = [Document(text = text)]
            nodes = splitter.get_nodes_from_documents(documents)
            for i,node in enumerate(nodes):
                if i==0:
                    child_chunks[f"{parent_chunk['ID']}_{i}"]=node.text
                elif i>0:
                    if len(node.text.split()) > 50:
                        child_chunks[f"{parent_chunk['ID']}_{i}"]=node.text
                    elif node.text not in child_chunks[list(child_chunks)[-1]]:
                        child_chunks[list(child_chunks)[-1]]+=node.text

        CHUNK_ID+=1
        all_chunks_processed.append({"Metadata":metadata,"ParentChunk":parent_chunk,"ChildChunks":child_chunks}) 
        
    return all_chunks_processed



def main():
    for ticker, company in TICKERS.items():
        for year in years:
            for quarter in quarters:
                curr_path = base_path.format(ticker=ticker,year=year,quarter=quarter)
                data = None
                with open(curr_path,'r') as f:
                    data = json.load(f)
                
                all_chunks_processed = post_processing_chunks(data,company,year,quarter)
                
                transcript = {
                    "ticker":ticker,
                    "quarter":quarter,
                    "year":year,
                    "transcript":all_chunks_processed
                }
                all_transcripts.append(transcript)
                print(f"Processed for {ticker} and {year+quarter}")

if __name__ == "__main__":
    main()

Processed for AAPL and 2024Q1
Processed for AAPL and 2024Q2
Processed for AAPL and 2024Q3
Processed for AAPL and 2025Q1
Processed for AAPL and 2025Q2
Processed for AAPL and 2025Q3
Processed for MSFT and 2024Q1
Processed for MSFT and 2024Q2
Processed for MSFT and 2024Q3
Processed for MSFT and 2025Q1
Processed for MSFT and 2025Q2
Processed for MSFT and 2025Q3
Processed for GOOGL and 2024Q1
Processed for GOOGL and 2024Q2
Processed for GOOGL and 2024Q3
Processed for GOOGL and 2025Q1
Processed for GOOGL and 2025Q2
Processed for GOOGL and 2025Q3
Processed for AMZN and 2024Q1
Processed for AMZN and 2024Q2
Processed for AMZN and 2024Q3
Processed for AMZN and 2025Q1
Processed for AMZN and 2025Q2
Processed for AMZN and 2025Q3
Processed for META and 2024Q1
Processed for META and 2024Q2
Processed for META and 2024Q3
Processed for META and 2025Q1
Processed for META and 2025Q2
Processed for META and 2025Q3
Processed for NVDA and 2024Q1
Processed for NVDA and 2024Q2
Processed for NVDA and 2024Q3
Proc

## SAVE ALL TRANSCRIPTS

In [7]:
with open(save_path,'w',encoding="utf-8") as file:
    json.dump(all_transcripts,file,indent = 4)

In [8]:
CHUNK_ID

16014